## Librerías

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet

## Data

In [2]:
ruta_archivo_train = 'Data/trainReg.txt'
ruta_archivo_test = 'Data/testReg.txt'

train_tot = pd.read_csv(ruta_archivo_train, delimiter=',')
test_tot = pd.read_csv(ruta_archivo_test, delimiter=',')


x_train_ori = train_tot[train_tot.columns[~train_tot.columns.isin(["V1", "V4"])]]
y_train_ori =train_tot["V1"]

x_test_ori =test_tot[test_tot.columns[~test_tot.columns.isin(["Id", "V4"])]]

## Train and Test

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x_train_ori, y_train_ori, test_size=0.3, random_state=100)

In [4]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((54445, 89), (23334, 89), (54445,), (23334,))

## Scale

In [5]:
# Scale
scaler = StandardScaler()
scaler.fit(x_train)

x_train_scale = scaler.transform(x_train)
x_test_scale = scaler.transform(x_test)
x_test_scale_ori = scaler.transform(x_test_ori)

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_spar

## Variables de Timbre y Covarianza

In [6]:
# Separar variables de timbre y covarianza
x_train_scale_timbre = x_train_scale[:, 0:11]
x_train_scale_covarianza = x_train_scale[:, 11:]

x_test_scale_timbre = x_test_scale[:, 0:11]
x_test_scale_covarianza = x_test_scale[:, 11:]

x_test_scale_timbre_ori = x_test_scale_ori[:, 0:11]
x_test_scale_covarianza_ori = x_test_scale_ori[:, 11:]

## PCA - Timbre

In [8]:
# PCA al timbre
n_components = 11

pca_timbre = PCA(n_components=n_components)
pca_timbre.fit(x_train_scale_timbre)

x_train_pca_timbre = pca_timbre.transform(x_train_scale_timbre)
x_test_pca_timbre = pca_timbre.transform(x_test_scale_timbre)
x_test_pca_timbre_ori = pca_timbre.transform(x_test_scale_timbre_ori)

# ------------------------------------------------
# Comentarrrr
x_train_pca_timbre = (x_train_scale_timbre)
x_test_pca_timbre = (x_test_scale_timbre)
x_test_pca_timbre_ori = (x_test_scale_timbre_ori)
# ------------------------------------------------

# DataFrame
columns = [f"PC-T-{i}" for i in range(1, n_components + 1)]
x_train_pca_timbre_df = pd.DataFrame(x_train_pca_timbre, columns=columns)
x_test_pca_timbre_df = pd.DataFrame(x_test_pca_timbre, columns=columns)
x_test_pca_timbre_ori_df = pd.DataFrame(x_test_pca_timbre_ori, columns=columns)

## PCA - Covarianza

In [9]:
# PCA al timbre
n_components = 78

pca_covarianza = PCA(n_components=n_components)
pca_covarianza.fit(x_train_scale_covarianza)

x_train_pca_covarianza = pca_covarianza.transform(x_train_scale_covarianza)
x_test_pca_covarianza = pca_covarianza.transform(x_test_scale_covarianza)
x_test_pca_covarianza_ori = pca_covarianza.transform(x_test_scale_covarianza_ori)

# DataFrame
columns = [f"PC-C-{i}" for i in range(1, n_components + 1)]
x_train_pca_covarianza_df = pd.DataFrame(x_train_pca_covarianza, columns=columns)
x_test_pca_covarianza_df = pd.DataFrame(x_test_pca_covarianza, columns=columns)
x_test_pca_covarianza_ori_df = pd.DataFrame(x_test_pca_covarianza_ori, columns=columns)

In [10]:
x_train_pca_df = pd.concat([x_train_pca_timbre_df, x_train_pca_covarianza_df], axis=1)
x_test_pca_df = pd.concat([x_test_pca_timbre_df, x_test_pca_covarianza_df], axis=1)
x_test_pca_ori_df = pd.concat([x_test_pca_timbre_ori_df, x_test_pca_covarianza_ori_df], axis=1)

In [20]:
# # Aplicar PCA a los datos de entrenamiento y prueba con 23 componentes
# n_components = 89

# pca = PCA(n_components=n_components)
# pca.fit(x_train_scale)

# x_train_pca = pca.transform(x_train_scale)
# x_test_pca = pca.transform(x_test_scale)
# x_test_pca_ori = pca.transform(x_test_scale_ori)
# # Convert NumPy arrays to Pandas DataFrames
# columns = [f"PC{i}" for i in range(1, n_components + 1)]
# x_train_pca_df = pd.DataFrame(x_train_pca, columns=columns)
# x_test_pca_df = pd.DataFrame(x_test_pca, columns=columns)
# x_test_pca_ori_df = pd.DataFrame(x_test_pca_ori, columns=columns)

## Modelo

In [17]:
# Create a Gradient Boosting Regressor model
model = ElasticNet(
    alpha=0.001,
    random_state=42,
    l1_ratio=0,
    # n_estimators=100,
    # max_features='sqrt',
    # max_depth=30,
    # min_samples_split=2,
    # min_samples_leaf=3,
)
# Train the model
model.fit(x_train_pca_df, y_train)

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: Conver

ElasticNet(alpha=0.001, l1_ratio=0, random_state=42)

In [18]:
y_pred_train = model.predict(x_train_pca_df)
y_pred_test = model.predict(x_test_pca_df)
mse_train = mean_squared_error(y_train, y_pred_train)
print("mse")
print(mse_train)
mse_test = mean_squared_error(y_test, y_pred_test)
print("mse")
print(mse_test)

mse
88.61382042403599
mse
89.74436115292131


c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_spar

In [23]:
y_pred_test_ori = model.predict(x_test_pca_ori_df)

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [24]:
# Create the results DataFrame
results = pd.DataFrame({"Id": test_tot["Id"], "y": y_pred_test_ori.flatten()})

# Print the first few rows of the results DataFrame
print(results.head())

# Guardar el DataFrame en un archivo CSV
# ruta_resultados = r'C:\Users\Usuario\Downloads\Statistical\Statistical\proy\Regre\boost11100.csv'

   Id            y
0   1  2003.900833
1   2  1997.359920
2   3  2000.168710
3   4  2002.754024
4   5  2004.414182


In [25]:
ruta_resultados = f'resultados_rf_100.csv'

results.to_csv(ruta_resultados, index=False)